In [92]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [136]:
def IQR_HD_cal(path):

    files = os.listdir(path)
    
    if 'B-ENaC' in path:
        healthy_files = [file for file in files if 'wt' in file]
        
    elif 'MPS' in path:
        df_mps = pd.read_csv('mouse_mps_summary.csv')
        df_mps_wt = df_mps[df_mps['Genotype'] == 'WT']
        scan_name = list(df_mps_wt['ScanName'].values)
        healthy_files = [str(name) + '.specificVentilation.csv' for name in scan_name]
        
    elif 'PA' in path:
        healthy_files = [file for file in files if 'WT' in file]
        
    elif 'Bead' in path:
        healthy_files = [file for file in files if 'WT' in file and 'beads' not in file]
        

    # calculate the average IQR for healthy littermate
    healthy_iqr_list = []

    for file in healthy_files:

        data = pd.read_csv(path + file, encoding="utf-8")

        data = data.to_numpy()

        sv_data = data[:,0]

        q1 = np.percentile(sv_data, 25)

        q3 = np.percentile(sv_data, 75)

        iqr = q3 - q1

        healthy_iqr_list.append(iqr)

    average_healthy_iqr = sum(healthy_iqr_list)/len(healthy_iqr_list)


    dic = dict()

    for file in files:

        data = pd.read_csv(path + file, encoding="utf-8")

        data = data.to_numpy()

        sv_data = data[:,0]

        q1 = np.percentile(sv_data, 25)

        q3 = np.percentile(sv_data, 75)

        iqr = q3 - q1

        dic[file] = [iqr, iqr/average_healthy_iqr]

    df = pd.DataFrame.from_dict(dic, orient='index', columns=['IQR', 'HD'])

    return df

In [128]:
df_b_enac = IQR_HD_cal("./Mouse B-ENaC Study/csv/")
df_b_enac.to_csv('IQR_HD_summary/Mouse_B-ENaC_HD.csv', index=True)

In [129]:
df_mps = IQR_HD_cal("./Mouse MPS Study/csv/")
df_mps.to_csv('IQR_HD_summary/Mouse_MPS_HD.csv', index=True)

In [130]:
df_pa = IQR_HD_cal("./Rat PA Study/csv/")
df_pa.to_csv('IQR_HD_summary/Rat_PA_HD.csv', index=True)

In [137]:
df_bead = IQR_HD_cal("./Rat Sterile Bead Study/csv/baseline_and_post_beads/")
df_bead.to_csv('IQR_HD_summary/Rat_Sterile_Bead_HD.csv', index=True)